<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

# Import e instalação das bibliotecas necessárias
Execute este bloco antes de todos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Install e downloads
# !pip install emoji
# !pip install pandas
# !pip install nltk
#nltk.download('all-nltk') #Demora um pouco

In [3]:
# Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools
import sklearn
from sklearn.model_selection import train_test_split
import math

# Padding
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Carrega dicionários, constantes e definições para a classificação
Execute esse bloco antes dos de classificação e balanceamento

In [4]:
# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú]+'

########## PATHS DE ARQUIVOS ##########

# Base completa bruta (Base 0)
PATH_BASE_COMPLETA_BRUTA = r"/content/drive/MyDrive/TCC/dados/tweets_coletados/base80k.csv"

# Base após pré-processamento COM stopwords (Base 1)
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base após pré-processamento SEM stopwords (Base 2)
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Amostra Base 1
PATH_TESTE_BASE1 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base1_Com_StopWords.csv"  

# Amostra Base 2
PATH_TESTE_BASE2 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

# Lista de Palavrões
PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

In [5]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

,hatelist
0,aberração
1,abestado
2,abestalhado
3,aborto
4,acabouamamata


In [6]:
# Carrega dicionário badword
dict_badword = pd.read_csv(PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR)
dict_badword.head(5)

,badword_list
0,babaca
1,boceta
2,boquete
3,bosseta
4,buceta


In [7]:
# Definição da função de classificação

def classifica(tweet, dicionario, LABEL1, LABEL2):
  # Tokeniza o tweet  
  tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
  tokens = tokenizer.tokenize(tweet)
  
  chave = dicionario.keys()[0]

  lista = list(dicionario[chave])
  lista_expressoes = []

  # Gera lista de expressões
  for termo in lista:
    termo_tokens = tokenizer.tokenize(termo)
    if(len(termo_tokens)>1): #expressoes
      lista_expressoes.append(termo)

  # Verifica se o tweet possui ao menos uma PALAVRA de hate
  for token in tokens:
    if token in lista:
      return LABEL1

  # Verifica se o tweet possui ao menos uma EXPRESSÃO de hate
  for palavra in lista_expressoes:
    result = re.subn(r'^' + palavra + '\s', palavra + ' ', tweet) # Substitui no início
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '$',' '+ palavra, tweet) # Substitui no final
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '\s', ' '+palavra+ ' ', tweet) #Substitui no meio
    if(result[1]):
      break

  if(result[1]):
    return LABEL1
  else:
    return LABEL2

    

# Classificar Base 1 e Base 2

In [ ]:
#Carrega a base de dados com stopWords
base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
#base1 = pd.read_csv(PATH_TESTE_BASE1, index_col=0)
base1.head()

,text
1,retwet eu queria compartilhar mas antes eu gos...
2,gostei da pulseira e da educação dele nossaaa ...
3,esses não irão ao encontro e beijing
4,cadê eles está faltando
5,e eu me questionando quem é esse blogueiro sem...


In [ ]:
#Carrega a base de dados sem stopWords
base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
#base2= pd.read_csv(PATH_TESTE_BASE2, index_col=0)
base2.head()

,text
1,retwet queria compartilhar antes gostaria sabe...
2,gostei pulseira educação nossaaa muita elegânc...
3,irão encontro beijing
5,questionando é blogueiro blog juntos
6,renan caloteiro desonesto vejam link abaixo


In [ ]:
base1.shape

(73462, 1)

In [ ]:
base2.shape

(64847, 1)

In [ ]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 1
base1['label'] = base1['text'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base1.head()

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate


In [ ]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 2
base2['label'] = base2['text'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base2.head()

,text,label
1,retwet queria compartilhar antes gostaria sabe...,not_hate
2,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,irão encontro beijing,not_hate
5,questionando é blogueiro blog juntos,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate


In [ ]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base1['classificacao_badword'] = base1['text'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base1.head()

,text,label,classificacao_badword
1,retwet eu queria compartilhar mas antes eu gos...,not_hate,not_badword
2,gostei da pulseira e da educação dele nossaaa ...,not_hate,not_badword
3,esses não irão ao encontro e beijing,not_hate,not_badword
4,cadê eles está faltando,not_hate,not_badword
5,e eu me questionando quem é esse blogueiro sem...,not_hate,not_badword


In [ ]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base2['classificacao_badword'] = base2['text'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base2.head()

,text,label,classificacao_badword
1,retwet queria compartilhar antes gostaria sabe...,not_hate,not_badword
2,gostei pulseira educação nossaaa muita elegânc...,not_hate,not_badword
3,irão encontro beijing,not_hate,not_badword
5,questionando é blogueiro blog juntos,not_hate,not_badword
6,renan caloteiro desonesto vejam link abaixo,not_hate,not_badword


In [ ]:
# BASE 1 COM STOP WORDS
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop1 = base1[ (base1['classificacao_badword']=='badword') & (base1['label']=='not_hate')].index 
base1.drop(baseDrop1, inplace=True)

# Dropa colunas que não serão utilizadas
base1.drop(['classificacao_badword'], axis='columns', inplace=True)

In [ ]:
base1

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
...,...,...
82244,abençoado ano novo sua linda e querida obrigad...,not_hate
82245,leva eles com a gente eles são amorzinhos d,not_hate
82246,k sai daqui marivaldo tu me odeia né pelo jeito,hate
82247,se tu nunca fumou maconha cole isso no seu mur...,not_hate


In [ ]:
# BASE 2 SEM STOP WORDS
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop2= base2[ (base2['classificacao_badword']=='badword') & (base2['label']=='not_hate')].index 
base2.drop(baseDrop2, inplace=True)

# Dropa colunas que não serão utilizadas
base2.drop(['classificacao_badword'], axis='columns', inplace=True)

In [ ]:
base2

,text,label
1,retwet queria compartilhar antes gostaria sabe...,not_hate
2,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,irão encontro beijing,not_hate
5,questionando é blogueiro blog juntos,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate
...,...,...
82242,egoísmo é grande parte produto sociedade émile...,not_hate
82244,abençoado novo linda querida obrigada carinho ...,not_hate
82245,leva gente amorzinhos d,not_hate
82246,k sai daqui marivaldo odeia né jeito,hate


# Classificar Base 0 (não processada, base bruta)

In [ ]:
#Carrega a base de dados bruta
base0 = pd.read_csv(PATH_BASE_COMPLETA_BRUTA, index_col=0)
# Dropa colunas que não serão utilizadas
base0.drop(['id_str', 'created_at','entities.hashtags','place.full_name'], axis='columns', inplace=True)
base0.head()

,text
0,@allantercalivre PARABÉNS @allantercalivre
1,RT @franciscoedi11: @allantercalivre Eu queria...
2,@allantercalivre @EdnaBraga Gostei da pulseira...
3,@allantercalivre @FBI Esses não irão ao encont...
4,@allantercalivre Cadê eles ?\r\nTá faltando


In [ ]:
base0.shape

(82149, 1)

In [ ]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 0
base0['label'] = base0['text'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base0.head()

,text,label
0,@allantercalivre PARABÉNS @allantercalivre,not_hate
1,RT @franciscoedi11: @allantercalivre Eu queria...,not_hate
2,@allantercalivre @EdnaBraga Gostei da pulseira...,not_hate
3,@allantercalivre @FBI Esses não irão ao encont...,not_hate
4,@allantercalivre Cadê eles ?\r\nTá faltando,not_hate


In [ ]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 0
base0['classificacao_badword'] = base0['text'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base0.head()

,text,label,classificacao_badword
0,@allantercalivre PARABÉNS @allantercalivre,not_hate,not_badword
1,RT @franciscoedi11: @allantercalivre Eu queria...,not_hate,not_badword
2,@allantercalivre @EdnaBraga Gostei da pulseira...,not_hate,not_badword
3,@allantercalivre @FBI Esses não irão ao encont...,not_hate,not_badword
4,@allantercalivre Cadê eles ?\r\nTá faltando,not_hate,not_badword


In [ ]:
# BASE 0 NÃO PROCESSADA
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop0 = base0[ (base0['classificacao_badword']=='badword') & (base0['label']=='not_hate')].index 
base0.drop(baseDrop0, inplace=True)

# Dropa colunas que não serão utilizadas
base0.drop(['classificacao_badword'], axis='columns', inplace=True)

In [ ]:
base0

,text,label
0,@allantercalivre PARABÉNS @allantercalivre,not_hate
1,RT @franciscoedi11: @allantercalivre Eu queria...,not_hate
2,@allantercalivre @EdnaBraga Gostei da pulseira...,not_hate
3,@allantercalivre @FBI Esses não irão ao encont...,not_hate
4,@allantercalivre Cadê eles ?\r\nTá faltando,not_hate
...,...,...
82244,@Mary_Greicy Abençoado Ano Novo sua linda e q...,not_hate
82245,"@MarcusConcolato leva eles com a gente, eles s...",not_hate
82246,"@marivildiz KKKKKKKKK sai daqui Marivaldo, tu ...",hate
82247,se tu NUNCA fumou maconha cole isso no seu mur...,not_hate


# Definições balanceamento
Execute esse bloco antes de balancear alguma base


In [8]:
categorias_desejadas = ['hate', 'not_hate'] # !!!

def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

# Balanceia BASE 1

In [ ]:
# Agrupa dados por label
base1.groupby('label').count()

,text
label,
hate,5575
not_hate,64183


In [ ]:
df_filtrado1 = base1[base1.label.isin(categorias_desejadas)]
df_filtrado1.shape

(69758, 2)

In [ ]:
df_filtrado1

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
...,...,...
82244,abençoado ano novo sua linda e querida obrigad...,not_hate
82245,leva eles com a gente eles são amorzinhos d,not_hate
82246,k sai daqui marivaldo tu me odeia né pelo jeito,hate
82247,se tu nunca fumou maconha cole isso no seu mur...,not_hate


In [ ]:
# Aplica função de balancear os dados
dataBC1 = balanceia_dados(df_filtrado1)

# DATA BALANCEADO
dataBC1.reset_index(drop=True, inplace=True) # reseta index
dataBC1.shape

(11150, 2)

In [ ]:
# convert output  hate =1 not_hate =0
dataBC1.label = [1 if each == "hate" else 0 for each in dataBC1.label] 

In [ ]:
dataBC1.groupby('label').count()

,text
label,
0,5575
1,5575


In [ ]:
# Salva a base1 classificada e balanceada

dataBC1.to_csv("/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1_classificada_balanceada.csv") 

# Balanceia BASE 2

In [ ]:
# Agrupa dados por label
base2.groupby('label').count()

,text
label,
hate,5287
not_hate,56160


In [ ]:
df_filtrado2 = base2[base2.label.isin(categorias_desejadas)]
df_filtrado2.shape

(61447, 2)

In [ ]:
df_filtrado2

,text,label
1,retwet queria compartilhar antes gostaria sabe...,not_hate
2,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,irão encontro beijing,not_hate
5,questionando é blogueiro blog juntos,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate
...,...,...
82242,egoísmo é grande parte produto sociedade émile...,not_hate
82244,abençoado novo linda querida obrigada carinho ...,not_hate
82245,leva gente amorzinhos d,not_hate
82246,k sai daqui marivaldo odeia né jeito,hate


In [ ]:
# Aplica função de balancear os dados
dataBC2 = balanceia_dados(df_filtrado2)

# DATA BALANCEADO
dataBC2.reset_index(drop=True, inplace=True) # reseta index
dataBC2.shape

(10574, 2)

In [ ]:
# convert output  hate =1 not_hate =0
dataBC2.label = [1 if each == "hate" else 0 for each in dataBC2.label] 

In [ ]:
dataBC2.groupby('label').count()

,text
label,
0,5287
1,5287


In [ ]:
# Salva a base2 classificada e balanceada
dataBC2.to_csv("/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv")

# Balanceia BASE 0

In [ ]:
# Agrupa dados por label
base0.groupby('label').count()

In [ ]:
df_filtrado0 = base0[base0.label.isin(categorias_desejadas)]
df_filtrado0.shape

In [ ]:
df_filtrado0

In [ ]:
# Aplica função de balancear os dados
dataBC0 = balanceia_dados(df_filtrado0)

# DATA BALANCEADO
dataBC0.reset_index(drop=True, inplace=True) # reseta index
dataBC0.shape

In [ ]:
# convert output  hate =1 not_hate =0
dataBC0.label = [1 if each == "hate" else 0 for each in dataBC0.label] 

In [ ]:
dataBC0.groupby('label').count()

In [ ]:
# Salva a base0 classificada e balanceada
dataBC0.to_csv("/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base0_classificada_balanceada.csv")

# Definições text padding
Executar esse bloco antes de dividir as bases

In [9]:
def tokenize(tweets_list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets_list)
    return tokenizer.texts_to_sequences(tweets_list), tokenizer

In [10]:
def pad(text_tokenized, length=None):
    return pad_sequences(text_tokenized, maxlen=length, padding='post')

In [11]:
def preprocess(tweets_list, max_text_length):
   
    preprocess_tweets_list, tweets_list_tokenizer = tokenize(tweets_list)

    preprocess_tweets_list = pad(preprocess_tweets_list, length=max_text_length)

    return preprocess_tweets_list, tweets_list_tokenizer

# Divisão manual das bases (0, 1 e 2) em treino(60%), teste(20%) e validação(20%)

In [13]:
# Base0 NÃO Processada
PATH_BASE0_NAO_PROCESSADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base0_classificada_balanceada.csv"

# Base1 Com StopWords
PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base1JuntaHateENotHateDaLeila_balanceada.csv"

# Base2 Sem StopWords
PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_maior4_menor25_limpissima_balanceada.csv"

# NOSSA Base2 Sem StopWords 
PATH_BASE_2_CLASSIFICADA_BALANCEADA = "/content/drive/MyDrive/TCC/dados/processadas/balanceadas/Base2_classificada_balanceada.csv"

NUM_BASES = 4

In [14]:
# Leitura das bases
tweets_base0 = pd.read_csv(PATH_BASE0_NAO_PROCESSADA, index_col=0)
print(tweets_base0.shape)
tweets_base1 = pd.read_csv(PATH_BASE1_JUNTO_COM_DA_LEILA_BALANCEADA, index_col=0)
print(tweets_base1.shape)
tweets_base2 = pd.read_csv(PATH_BASE2_LEILA_LIMPISSIMA_BALANCEADA, index_col=0)
print(tweets_base2.shape)
tweets_base3 = pd.read_csv(PATH_BASE_2_CLASSIFICADA_BALANCEADA, index_col=0)
print(tweets_base3.shape)

bases = [tweets_base0, tweets_base1, tweets_base2, tweets_base3]

(9274, 2)
(16036, 2)
(4886, 2)
(10574, 2)


In [15]:
# Gera as quantidades das divisões de cada base (train 60%, test 20% e validation 20%)
list_test_size = []
list_train_size = [] # Só para verificação, não é usado posteriormente
list_val_size = []
input_datas = []
output_labels = []
for base in bases:
  val_size = base.shape[0]/5 # 20%
  val_size = int(math.ceil(val_size))

  test_size = base.shape[0]/5 # 20%
  test_size = int(math.ceil(test_size))

  train_size = base.shape[0] - (test_size+val_size)
  list_test_size.append(test_size)
  list_train_size.append(train_size)
  list_val_size.append(val_size)

  input_data, _ = preprocess(base['text'], None)
  input_datas.append(input_data)
  output_labels.append(base['label'])

print('list_train_size: '+str(list_train_size))
print('list_test_size: '+str(list_test_size))
print('list_val_size: '+str(list_val_size))

list_train_size: [5564, 9620, 2930, 6344]
list_test_size: [1855, 3208, 978, 2115]
list_val_size: [1855, 3208, 978, 2115]


In [17]:
# Divide cada base em 3 fatias (train, test e val). Para cada fatia temos os dados X e as labels Y
list_x_train = []
list_y_train = []
list_x_test = []
list_y_test = []
list_x_val = []
list_y_val = []
for i in range(NUM_BASES):
  x_main, x_test, y_main, y_test = train_test_split(input_datas[i], output_labels[i], test_size=list_test_size[i], stratify=output_labels[i], random_state=42)

  x_train, x_val, y_train, y_val = train_test_split(x_main, y_main, test_size=list_val_size[i], stratify=y_main, random_state=42)

  list_x_train.append(x_train)
  list_y_train.append(y_train)
  list_x_test.append(x_test)
  list_y_test.append(y_test)
  list_x_val.append(x_val)
  list_y_val.append(y_val)

In [18]:
# Gera 3 dataframes para cada base e salva em um dicionário
# dict_output_files {0: df_train, df_test, df_val, 1: df_train, df_test, df_val, 2: df_train, df_test, df_val}
dict_output_files = {}
for i in range(NUM_BASES):
  df_train = pd.DataFrame(list_x_train[i])
  df_train['label'] = list_y_train[i].reset_index(drop=True)

  df_test = pd.DataFrame(list_x_test[i])
  df_test['label'] = list_y_test[i].reset_index(drop=True)

  df_val = pd.DataFrame(list_x_val[i])
  df_val['label'] = list_y_val[i].reset_index(drop=True)

  dict_output_files[i] = [df_train, df_test, df_val]

dict_output_files

{0: [          0      1     2     3      4     5      6     7    8     9  ...  63  \
  0        12    108   374  7291     54     0      0     0    0     0  ...   0   
  1        12     84    55     1    564   100    728     0    0     0  ...   0   
  2         8      6   138   678     18    22    154   138    6    68  ...   0   
  3        84      2   115    87    326  7787     76    25    5   795  ...   0   
  4      7356  18209    12    92     50    61    139  2520    0     0  ...   0   
  ...     ...    ...   ...   ...    ...   ...    ...   ...  ...   ...  ...  ..   
  5559     62   3864   175  3111   2306   216    157     2  216  2307  ...   0   
  5560    354    413   400    12     74    14     75  2408   28    12  ...   0   
  5561     31   6398     1  1564      1   293     64  1821  906    65  ...   0   
  5562  17107   2232  2286     1   5057   302  17108     0    0     0  ...   0   
  5563    211     30    39    31  22854  2503      0     0    0     0  ...   0   
  
        64

In [19]:
dict_output_files[3][0].groupby('label').count()[0]

label
0    3172
1    3172
Name: 0, dtype: int64

In [23]:
dict_output_files[3][1].groupby('label').count()[0]

label
0    1058
1    1057
Name: 0, dtype: int64

In [24]:
dict_output_files[3][2].groupby('label').count()[0]

label
0    1057
1    1058
Name: 0, dtype: int64

In [22]:
# Salva os dataframes em arquivos. 3 arquivos para cada base
for i in range(NUM_BASES):
  dict_output_files[i][0].to_csv('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/train'+str(i)+'.csv')
  dict_output_files[i][1].to_csv('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/test'+str(i)+'.csv')
  dict_output_files[i][2].to_csv('/content/drive/MyDrive/TCC/dados/processadas/bases_balanceadas_divididas/val'+str(i)+'.csv')
